In [ ]:
import numpy as np
import sys
import json
from matplotlib import pyplot as plt
import meshplot as mp
import cv2
import svgwrite
import igl
sys.path.append("../src")

# 1. BFGS Approximated Hessian Matirx and its Inverse Matrix

In [ ]:
def f(x):
    return np.linalg.norm(x) ** 2

def grad_f(x):
    return 2 * x

In [ ]:
from bfgs import compute_approximate_hessian_matrix
x = np.array([1, 1])
p = -grad_f(x)
alpha = 1.0
y = x + alpha * p
sk = (y - x).reshape(2, 1)
yk = (grad_f(y) - grad_f(x)).reshape(2, 1)
Bk = np.identity(2)
Bnew = compute_approximate_hessian_matrix(sk, yk, Bk)

In [ ]:
from bfgs import compute_inverse_approximate_hessian_matrix
x = np.array([1, 1])
p = -grad_f(x)
alpha = 1.0
y = x + alpha * p
sk = (y - x).reshape(2, 1)
yk = (grad_f(y) - grad_f(x)).reshape(2, 1)
invBk = np.identity(2)
invBnew = compute_inverse_approximate_hessian_matrix(sk, yk, invBk)


The matrix multiplation of the two matrix should be indentity matrix.

In [ ]:
print(Bnew @ invBnew)

# 2. Gradient Descent with Backtracking Line Search 

In [ ]:
with open('../data/dinosaur.json', 'r') as infile:
    [V, F] = json.load(infile)
    V = np.array(V)
    F = np.array(F)

In [ ]:
from geometry import compute_mesh_centroid

shader = {"flat":True, # Flat or smooth shading of triangles
           "wireframe": True, "wire_width": 0.03, "wire_color": "black", # Wireframe rendering
           "width": 600, "height": 600, # Size of the viewer canvas
           "antialias": True, # Antialising, might not work on all GPUs
           "scale": 2.0, # Scaling of the model
           "side": "DoubleSide", # FrontSide, BackSide or DoubleSide rendering of the triangles
           "colormap": "viridis", "normalize": [None, None], # Colormap and normalization for colors
           "background": "#ffffff", # Background color of the canvas
           "line_width": 1.0, "line_color": "black", # Line properties of overlay lines
           "bbox": False, # Enable plotting of bounding box
           "point_color": "red", "point_size": 0.01 # Point properties of overlay points
          }

def draw_meshes_comparison(V1, F1, V2, F2, color1, color2):
    mesh_plot = mp.plot(V1, F1, c = color1, return_plot=True)
    mesh_plot.add_mesh(V2, F2, c = color2, shading=shader)
    
    center0 = np.array(compute_mesh_centroid(V2, F2))
    center1 = center0.copy()
    center1[1] = 0
    vertices = np.vstack([center0, center1])
    mesh_plot.add_points(vertices, shading={"point_color": "black", "point_size": 0.1})
    mesh_plot.add_edges(vertices, np.array([[0, 1]]), shading={"line_color": "black", "line_width" : 0.5});

In [ ]:
from geometry import compute_center_support_line
from linesearch import gradient_descent_with_line_search
x_csl = compute_center_support_line(V)
[V1, F, objective, running_time] = gradient_descent_with_line_search(V, F, x_csl, w = 10, obj_tol = 1E-8, theta = 1.0, beta = 0.5, c = 0.02, iter = 10000)

mesh_plot = draw_meshes_comparison(V, F, V1, F, np.array([0.3, 0.3, 0.3]), np.array([1.0, 1.0, 0]))
print(running_time)

In [ ]:
fig = plt.figure()
plt.plot(objective)
plt.xlabel("number of iteration")
plt.ylabel("objective energy")
plt.show()

# 3. BGFS with Backtracking Line Search 

In [ ]:
from geometry import compute_center_support_line
from bfgs import bfgs_with_line_search
x_csl = compute_center_support_line(V)
[V1, F, objective, running_time] = bfgs_with_line_search(V, F, x_csl, w = 10, obj_tol = 1E-8, theta = 1.0, beta = 0.5, c = 0.02, iter = 10000)

mesh_plot = draw_meshes_comparison(V, F, V1, F, np.array([0.3, 0.3, 0.3]), np.array([1.0, 1.0, 0]))
print(running_time)

In [ ]:
fig = plt.figure()
plt.plot(objective)
plt.xlabel("number of iteration")
plt.ylabel("objective energy")
plt.show()

# 4. Create your own Mesh

# 4.1 Load the image

In [ ]:
#read the image
image_path = "../data/dance.png"
img = cv2.imread(image_path, 1) 

# convert the image to grey scale
imgrey = 255 - cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# show image
plt.figure(figsize = (10, 10))
plt.imshow(imgrey)
plt.show()

# 4.2 Make Flat Bottom Line

In [ ]:
#cut from Y = maxX
maxX = 940
imcut = imgrey.copy()
imcut[maxX : imcut.shape[0] + 1, :] = 0

# show image
plt.figure(figsize = (10, 10))
plt.imshow(imcut)
plt.show()

# 4.3 Extract and Simplify the Boundary Curves

To simplfy the boundary curves of the final mesh, we collapse the edge if its edge length is smaller than edge_length_tot.

In [ ]:
edge_length_tot = 10.0

In [ ]:
# Extrac Boundary
ret, thresh = cv2.threshold(imcut, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
plt.figure(figsize = (10, 10))
imContour = np.zeros(imcut.shape)

# Compute Point With Highest Y coordinate
maxY = 0
for k, _ in enumerate(contours):
    current_points = contours[k][:, 0]
    for pt in current_points:
        maxY = max(pt[1], maxY)
        
boundary_points = []

# draw countour
for k, _ in enumerate(contours):
    trimmed_points = []
    current_points = contours[k][:, 0]
    prev_pt = np.array(current_points[-1])
    for pt in current_points:
        curr_pt = np.array(pt)
        if np.linalg.norm(prev_pt - curr_pt) > edge_length_tot or pt[1] == maxY:
            prev_pt = curr_pt
            trimmed_points.append([pt[0], pt[1]])
            
    contours[k] = np.array([trimmed_points])
    if contours[k].shape[1] > 0:
        imContour = cv2.drawContours(imContour, contours, k, (255, 255, 255), 6)
        boundary_points.append(trimmed_points)

        
# scale boundary points
scale = max(imcut.shape[0], imcut.shape[1])
for pts in boundary_points:
    for pt in pts:
        pt[0] /= scale
        pt[1] = (maxY - pt[1]) / scale
        
plt.imshow(imContour)

# 4.4 Find Holes

Some curves stored in boundary_points refers to holes in the mesh, we put their indices in to holes_indices array to compute the centroid of the holes for late remeshing. If the image has no hole, just set holes_indices to be empty list.

In [ ]:
holes_indices = [1]

In [ ]:
Holes =  []
for k in holes_indices:
    ct = np.array([0, 0], dtype = float)
    for pt in boundary_points[k]:
        ct += np.array(pt, dtype = float)
    ct /= len(boundary_points[k])
    Holes.append([ct[0], ct[1]])

# 4.5 Draw Mesh

In [ ]:
from geometry import compute_mesh_centroid, compute_center_support_line
def draw_mesh(V, F):
    shading = {"flat":True, # Flat or smooth shading of triangles
           "wireframe":True, "wire_width": 0.03, "wire_color": "black", # Wireframe rendering
           "width": 600, "height": 600, # Size of the viewer canvas
           "antialias": True, # Antialising, might not work on all GPUs
           "scale": 2.0, # Scaling of the model
           "side": "DoubleSide", # FrontSide, BackSide or DoubleSide rendering of the triangles
           "colormap": "viridis", "normalize": [None, None], # Colormap and normalization for colors
           "background": "#ffffff", # Background color of the canvas
           "line_width": 1.0, "line_color": "black", # Line properties of overlay lines
           "bbox": False, # Enable plotting of bounding box
           "point_color": "red", "point_size": 0.01 # Point properties of overlay points
          }
    p = mp.plot(V, F, shading=shading, return_plot=True)
    center0 = np.array(compute_mesh_centroid(V, F))
    center1 = center0.copy()
    center1[1] = 0
    vertices = np.vstack([center0, center1])
    p.add_points(vertices, shading={"point_color": "black", "point_size": 0.1})
    p.add_edges(vertices, np.array([[0, 1]]), shading={"line_color": "black", "line_width" : 0.5});

In [ ]:
from utility import triangulate_mesh
[V, F] = triangulate_mesh(boundary_points, Holes, 0.001)
draw_mesh(V, F)

# 4.6 Save as SVG

The following code is for outputing the svg files.

The unit in the following svg file is mm. A point (100, 100) refers to a point (10 cm, 10 cm).

You have to write code to generate slits by yourself to provide off-plane stability for your laser cutting model.

In [ ]:
dwg = svgwrite.Drawing('dance.svg')
for Es in igl.boundary_facets(F):
    p0 = V[Es[0]]
    p1 = V[Es[1]]
    dwg.add(dwg.line((p0[0], p0[1]), (p1[0], p1[1])))
dwg.save()

# 5. Design your own Mesh

Please design 3 different meshes that are original not equilibrium.

Try to optimize them to be equilibrium and report it in your pdf handin.